In [9]:
import numpy as np
import random
import time

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(y):
    return y * (1.0 - y)

In [10]:
class FFN:
    def __init__(self, input, hidden, output, iterations, learning_rate, rate_decay):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.rate_decay = rate_decay

        self.input = input + 1
        self.hidden = hidden
        self.output = output
        self.ai = [1.0 for _ in range(self.input)]
        self.ah = [1.0 for _ in range(self.hidden)]
        self.ao = [1.0 for _ in range(self.output)]

        input_range = 1.0 / self.input ** (1/2)
        output_range = 1.0 / self.hidden ** (1/2)
        self.wi = np.random.normal(loc=0, scale=input_range, size = (self.input, self.hidden))
        self.wo = np.random.normal(loc=0, scale=output_range, size = (self.hidden, self.output))

        self.ci = np.zeros((self.input, self.hidden))
        self.co = np.zeros((self.hidden, self.output))

    def feedForward(self, inputs):
        if len(inputs) != self.input-1:     # 最后一项作为bias
            raise ValueError('Wrong number of inputs you silly goose!')
        for i in range(self.input - 1):
            self.ai[i] = inputs[i]

        for j in range(self.hidden):
            sum = 0.0
            for i in range(self.input):
                sum += self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(sum)
        
        for k in range(self.output):
            sum = 0.0
            for j in range(self.hidden):
                sum += self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(sum)
        
        return self.ao[:]
    
    def backPropagate(self, targets):
        if len(targets) != self.output:
            raise ValueError('Wrong number of targets you silly goose!')
        
        output_deltas = [0.0] * self.output
        for k in range(self.output):
            error = -(targets[k] - self.ao[k])
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        hidden_deltas = [0.0] * self.hidden
        for j in range(self.hidden):
            error = 0
            for k in range(self.output):
                error += output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        for j in range(self.hidden):
            for k in range(self.output):
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] -= self.learning_rate * change + self.co[j][k]
                self.co[j][k] = change 

        for i in range(self.input):
            for j in range(self.hidden):
                change = hidden_deltas[j] * self.ai[i]
                self.wi[i][j] -= self.learning_rate * change + self.ci[i][j]
                self.ci[i][j] = change
        
        error = 0.0
        for k in range(len(targets)):
            error += 0.5 * (targets[k] - self.ao[k])**2
        return error


    #测试
    def test(self, patterns):
        """
        Currently this will print out the targets next to the predictions.
        Not useful for actual ML, just for visual inspection.
        """
        for p in patterns:
            print(p[1], '->', self.feedForward(p[0]))
    
    #训练
    def train(self, patterns):
        # N: learning rate
        for i in range(self.iterations):
            error = 0.0
            random.shuffle(patterns)
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.feedForward(inputs)
                error += self.backPropagate(targets)
            if i % 10 == 0:
                print("iterations:%d ,lr:%-.5f ,error:%-.5f " % (i,self.learning_rate,error))
            # with open('error.txt', 'a') as errorfile:
            #     errorfile.write(str(error) + '\n')
            #     errorfile.close()
            # if i % 10 == 0:
            #     print('error %-.5f' % error)
            # learning rate decay
            self.learning_rate = self.learning_rate * (self.learning_rate / (self.learning_rate + (self.learning_rate * self.rate_decay)))

    #预测
    def predict(self, X):
        """
        return list of predictions after training algorithm
        """
        predictions = []
        for p in X:
            predictions.append(self.feedForward(p))
        return predictions

In [11]:
def demo():
    """
    run NN demo on the digit recognition dataset from sklearn
    """
    def load_data():
        data = np.loadtxt('../data/sklearn_digits.csv', delimiter = ',')

        # first ten values are the one hot encoded y (target) values
        y = data[:,0:10] # 0-9 之间的数字
        
        data = data[:,10:] # x data
        data -= data.min() # scale the data so values are between 0 and 1
        data /= data.max() # scale
        
        out = []
        #print data.shape

        # populate the tuple list with the data
        for i in range(data.shape[0]):
            tupledata = list((data[i,:].tolist(), y[i].tolist())) # don't mind this variable name
            out.append(tupledata)

        return out
    
    start = time.time()
    
    X = load_data()

    #print X[9] # make sure the data looks right

    NN = FFN(64, 100, 10, iterations = 50, learning_rate = 0.5,  rate_decay = 0.01)

    NN.train(X)
    
    NN.test(X)
    
    end = time.time()
    print(end - start)

In [12]:
if __name__ == '__main__':
    demo()

iterations:0 ,lr:0.50000 ,error:374.24541 
iterations:10 ,lr:0.45264 ,error:16.69411 
iterations:20 ,lr:0.40977 ,error:5.84331 
iterations:30 ,lr:0.37096 ,error:3.35235 
iterations:40 ,lr:0.33583 ,error:3.22234 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] -> [np.float64(8.586415656519723e-05), np.float64(3.3204208765439833e-09), np.float64(3.271719921490331e-07), np.float64(6.867970275556142e-09), np.float64(0.0012923879620822438), np.float64(3.164799344431513e-05), np.float64(0.9999701702476191), np.float64(9.85000999868901e-10), np.float64(0.0014606809529583114), np.float64(7.05844350360906e-14)]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] -> [np.float64(4.095405657378496e-09), np.float64(0.9987316630282699), np.float64(4.26967225429625e-05), np.float64(3.4304798790260865e-08), np.float64(7.039872682494538e-09), np.float64(6.375241841752985e-06), np.float64(5.320108385699993e-05), np.float64(1.4375886535736749e-11), np.float64(0.00038011379690048935), np.float64(7.82846

KeyboardInterrupt: 